In [ ]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

In [5]:
import yfinance as yf
import pandas as pd

# Create the ticker object for Tesla
tesla_ticker = yf.Ticker('TSLA')

# Extract Tesla stock history (max period)
tesla_data = tesla_ticker.history(period='max')

# Reset index for easier use later
tesla_data.reset_index(inplace=True)

# Display first rows
tesla_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [15]:
import requests
from bs4 import BeautifulSoup

# Tesla revenue URL
tesla_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

# Get page content
html_data = requests.get(tesla_url).text

# Parse HTML
soup = BeautifulSoup(html_data, 'html.parser')

# Create empty DataFrame
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# Select the correct revenue table; macrotrends has multiple tables, 'tbody' 1 index for quarterly data
tesla_revenue_table = soup.find_all('tbody')[1]

# Loop through rows and extract Date and Revenue
for row in tesla_revenue_table.find_all('tr'):
    cols = row.find_all('td')
    date = cols[0].text
    revenue = cols[1].text.replace('$', '').replace(',', '')  # clean revenue values
    new_row = {"Date": date, "Revenue": revenue}
    new_row_df = pd.DataFrame([new_row])
    tesla_revenue  = pd.concat([tesla_revenue, new_row_df], ignore_index=True)
# Drop empty or invalid rows
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

# Convert Revenue to numeric
tesla_revenue['Revenue'] = pd.to_numeric(tesla_revenue['Revenue'])

# Show last 5 rows
tesla_revenue.tail()


,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


In [16]:
# GameStop ticker
gme_ticker = yf.Ticker('GME')

# Extract stock history max period
gme_data = gme_ticker.history(period='max')

# Reset index for use
gme_data.reset_index(inplace=True)

# Display first rows
gme_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578048,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [22]:
!pip install html5lib

In [ ]:
import requests

# GameStop revenue URL
gme_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

# Get HTML content (not mandatory for pandas read_html but okay to keep)
gme_html = requests.get(gme_url).text

# Parse with BeautifulSoup (optional if only using pandas.read_html)
gme_soup = BeautifulSoup(gme_html, 'html.parser')

# Extract quarterly revenue table using pandas directly from URL
gme_revenue = pd.read_html(gme_url, match="GameStop Revenue", flavor='bs4')[0]

# Rename columns for clarity
gme_revenue = gme_revenue.rename(columns={
    "GameStop Quarterly Revenue(Millions of US $)": "Date",
    "GameStop Quarterly Revenue(Millions of US $).1": "Revenue"
})

# Clean revenue field removing $ and commas
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(',|\$', '', regex=True)

# Convert Revenue to numeric for analysis
gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"])

# Drop empty rows if any and show last 5 records
gme_revenue = gme_revenue[gme_revenue["Revenue"].notna()]

# Display last 5 rows of cleaned revenue data
print(gme_revenue.tail())


In [ ]:
import plotly.graph_objects as go

def make_graph(stock_data, revenue_data, company_name):
    fig = go.Figure()
    
    # Stock Price
    fig.add_trace(go.Scatter(
        x=stock_data['Date'], 
        y=stock_data['Close'], 
        mode='lines',
        name=f'{company_name} Stock Price'
    ))
    
    # Revenue (plotted on secondary y-axis)
    fig.add_trace(go.Scatter(
        x=revenue_data['Date'], 
        y=revenue_data['Revenue'], 
        mode='lines',
        name=f'{company_name} Revenue',
        yaxis='y2'
    ))
    
    # Layout configuration
    fig.update_layout(
        title=f'{company_name} Stock Price and Revenue Over Time',
        xaxis_title='Date',
        yaxis=dict(title='Stock Price (USD)', side='left'),
        yaxis2=dict(title='Revenue (Millions USD)', overlaying='y', side='right'),
        legend=dict(x=0, y=1.1, orientation='h')
    )
    
    fig.show()

# Format revenue date to datetime
tesla_revenue['Date'] = pd.to_datetime(tesla_revenue['Date'])
gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'])

# Plot Tesla dashboard
make_graph(tesla_data, tesla_revenue, 'Tesla')

# Plot GameStop dashboard
make_graph(gme_data, gme_revenue, 'GameStop')
